### network analysis
- 논문 간의 키워드 연관성을 추출해본다.
- 키워드로 논문간의 번들을 형성할 수 있다. 
    - 논문의 트렌드를 형성할 수 있다.

In [1]:
import numpy as np
import pandas as pd
from IPython.display import display
import re

In [2]:
df = pd.read_csv('references.csv')

In [3]:
df.tail()

,infor,refer
1457,KOSPI200 옵션전략지수의 성과\n권순신 | 강병진 | 정재만 | 한국파생상품학...,0
1458,KRX금시장과 국제금융시장의 상호연관성에 관한 연구 : CME선물시장을 중심으로\n...,0
1459,Analytical Approximations of American Call Opt...,0
1460,현금흐름 위험 기반 KOSPI 수익률 횡단면 연구\n문성제 | 송준혁 | 한국파생상...,0
1461,모호성 회피성향이 예방적 노력에 미치는 영향\n홍지민 | 한국파생상품학회 | 선물연...,0


### 참고문헌이 나와있지 않은 논문의 수

In [4]:
len([i for i in df['refer'].values if i == '0'])

242

In [5]:
str(((len([i for i in df['refer'].values if i == '0']) / len(df)) * 100)) + '%'

'16.55266757865937%'

# Tuning_process

#### 우선 information 컬럼을 늘려준다.
- refer 컬럼을 split 하는 과정을 시행한다.

In [6]:
print(len(re.findall('[0-9]{1,}. \[',df['refer'].values[100])))
print(len(re.split('[0-9]{1,}. \[',df['refer'].values[100])))

31
32


In [7]:
head_ls = re.findall('[0-9]{1,}. \[',df['refer'].values[100])

In [8]:
testing_ls = re.split('[0-9]{1,}. \[',df['refer'].values[100])
process_ls = []
for i in range(len(testing_ls)):
    if re.search('\/ [0-9]{,4} \/',testing_ls[i]) : 
        process_ls.append(testing_ls[i])
process_ls[-2:]

['학술지(정기간행물)] - / 1986 / Large Shareholders and Corporate Control (94) : 461 ~ 488,',
 '학술지(정기간행물)] - / 1995 / Do Corporations Award CEO Stock Options Effectively? 39 : 237 ~ 269']

In [9]:
title_ls = []
for i in range(len(process_ls)):
    title_ls.append(head_ls[i] + process_ls[i])
title_ls[-2:]

['30. [학술지(정기간행물)] - / 1986 / Large Shareholders and Corporate Control (94) : 461 ~ 488,',
 '31. [학술지(정기간행물)] - / 1995 / Do Corporations Award CEO Stock Options Effectively? 39 : 237 ~ 269']

### information 컬럼을 늘려주는 function(generalization) 프로세스

In [10]:
#length column 을 만들어주기 위한 프로세스
final_process_ls = []

for i in range(len(df['refer'])):
    try :
        title_ls = []
        if df['refer'].values[i] != '0' : 
            head_ls = re.findall('[0-9]{1,}. \[',df['refer'].values[i])
            testing_ls = re.split('[0-9]{1,}. \[',df['refer'].values[i])
            process_ls = []
            for j in range(len(testing_ls)):
                if re.search('\/ [0-9]{,4} \/',testing_ls[j]) : 
                    process_ls.append(testing_ls[j])
            for k in range(len(process_ls)):
                title_ls.append(head_ls[k] + process_ls[k])
        else : title_ls.append('0')
        final_process_ls.append(title_ls)
    except Exception as e:
        print(i)
        print(len(process_ls),len(head_ls))
        print(e)
        break

In [11]:
df['length'] = [len(i) for i in final_process_ls]

In [12]:
new_df = pd.DataFrame()
new_df['refer'] = [j for i in final_process_ls for j in i]

In [13]:
extending_title_ls = []
for i in range(len(df)):
    for _ in range(df['length'].values[i]):
        extending_title_ls.append(df['infor'].values[i])
print(len(extending_title_ls))
new_df['infor'] = extending_title_ls

37247


In [14]:
new_df = pd.DataFrame()
new_df['infor'] = extending_title_ls
new_df['refer'] = [j for i in final_process_ls for j in i]

In [15]:
set(new_df['infor'].values) == set(df['infor'].values)
#OK
new_df.tail()

,infor,refer
37242,KOSPI200 옵션전략지수의 성과\n권순신 | 강병진 | 정재만 | 한국파생상품학...,0
37243,KRX금시장과 국제금융시장의 상호연관성에 관한 연구 : CME선물시장을 중심으로\n...,0
37244,Analytical Approximations of American Call Opt...,0
37245,현금흐름 위험 기반 KOSPI 수익률 횡단면 연구\n문성제 | 송준혁 | 한국파생상...,0
37246,모호성 회피성향이 예방적 노력에 미치는 영향\n홍지민 | 한국파생상품학회 | 선물연...,0


In [16]:
new_df.to_csv('extended_references.csv',index=False)

# Text Tuning process

# 우선 references 가 아닌 논문의 이름부터 튜닝한다.

infor 컬럼은 | 단위로 잘라줄 수 있고 \n 단위로 잘라줄 수 있다.

In [17]:
for idx in range(len(new_df['infor'].values)):
    if '\n' not in new_df['infor'].values[idx].split('|')[0]:
        print(idx)

In [18]:
new_title_ls = []
for idx in range(len(new_df['infor'].values)):
    testing_ls = new_df['infor'].values[idx].split('|')
    new_title_ls.append([j for i in [i.split('\n') for i in testing_ls] for j in i])
len(new_title_ls) == len(new_df)

True

### 튜닝한 데이터에 대해서 해당 프로세스의 새로운 데이터들을 넣어줄 새로운 데이터 프레임 tuned_df 를 만들어준다.

In [19]:
tuned_df = pd.DataFrame()
tuned_df['title_bundle'] = new_title_ls
tuned_df.tail()

,title_bundle
37242,"[KOSPI200 옵션전략지수의 성과, 권순신 , 강병진 , 정재만 , 한국파..."
37243,"[KRX금시장과 국제금융시장의 상호연관성에 관한 연구 : CME선물시장을 중심으로,..."
37244,[Analytical Approximations of American Call Op...
37245,"[현금흐름 위험 기반 KOSPI 수익률 횡단면 연구, 문성제 , 송준혁 , 한국..."
37246,"[모호성 회피성향이 예방적 노력에 미치는 영향, 홍지민 , 한국파생상품학회 , ..."


### 리스트의 첫 번째 엘리먼트가 항상 논문의 이름인지 확인하는 프로세스가 필요해 보인다.

In [20]:
tuned_df['title_bundle'].values[0]

['기업의 소유구조와 기업가치 및 수익률에 대한 연구: 임원지분율을 중심으로',
 '김병호 ',
 ' 한국증권학회 ',
 ' Asia-Pacific Journal of Financial Studies ',
 ' 30() ',
 ' pp.391~434 ',
 ' 2002.06 ',
 ' 경영학',
 '피인용횟수 : 84']

#### 이름이 넉넉잡아 5글자 이내라고 가정, 검사해본다.

In [21]:
for paper in tuned_df['title_bundle'].values:
    if len(paper[0]) <= 5 : 
        print(paper)
        
# 없으므로 naive 하지만, 각 리스트의 첫 번째 엘리먼트는 페이퍼 이름인 것을 확인할 수 있다.

### paper name 파싱! 

In [22]:
tuned_df['paper_name'] = [i[0] for i in tuned_df['title_bundle'].values]
tuned_df.tail()

,title_bundle,paper_name
37242,"[KOSPI200 옵션전략지수의 성과, 권순신 , 강병진 , 정재만 , 한국파...",KOSPI200 옵션전략지수의 성과
37243,"[KRX금시장과 국제금융시장의 상호연관성에 관한 연구 : CME선물시장을 중심으로,...",KRX금시장과 국제금융시장의 상호연관성에 관한 연구 : CME선물시장을 중심으로
37244,[Analytical Approximations of American Call Op...,Analytical Approximations of American Call Opt...
37245,"[현금흐름 위험 기반 KOSPI 수익률 횡단면 연구, 문성제 , 송준혁 , 한국...",현금흐름 위험 기반 KOSPI 수익률 횡단면 연구
37246,"[모호성 회피성향이 예방적 노력에 미치는 영향, 홍지민 , 한국파생상품학회 , ...",모호성 회피성향이 예방적 노력에 미치는 영향


#### 리스트에서 두 번째부터 `한국증권학회` , `한국재무관리학회` , `한국파생상품학회` , `한국재무학회` 가 나오는 엘리먼트의 리스트 인덱스 전까지는 이름 즉, 저자이다.

In [23]:
tuned_df['title_bundle'].values[0]

['기업의 소유구조와 기업가치 및 수익률에 대한 연구: 임원지분율을 중심으로',
 '김병호 ',
 ' 한국증권학회 ',
 ' Asia-Pacific Journal of Financial Studies ',
 ' 30() ',
 ' pp.391~434 ',
 ' 2002.06 ',
 ' 경영학',
 '피인용횟수 : 84']

##### 우선 한국증권학회 , 한국재무관리학회 , 한국파생상품학회 , 한국재무학회 가 나오는 인덱스를 찾는다.

In [24]:
testing_ls = []
for paper in tuned_df['title_bundle'].values : 
    for idx in range(len(paper)):
        if re.findall('한국증권학회|한국재무관리학회|한국파생상품학회|한국재무학회',paper[idx]) : 
            testing_ls.append(idx)
            break

In [25]:
institution_ls = []
for idx in range(len(tuned_df['title_bundle'].values)):
    institution_ls.append(tuned_df['title_bundle'].values[idx][testing_ls[idx]])

In [26]:
institution_ls[-2:]

[' 한국파생상품학회 ', ' 한국파생상품학회 ']

In [27]:
for i in institution_ls : 
    if not re.findall('한국증권학회|한국재무관리학회|한국파생상품학회|한국재무학회',i):
        print(i)

# 아무것도 출력되지 않으므로 institution_ls 에는 발행기관명만 있다는 것을 알 수 있다.

In [28]:
tuned_df['institution'] = institution_ls

### 이번에는 이름을 해본다. 이름은 두번째 요소부터 발행기관명 사이의 엘리먼트이다. 발행기관명의 인덱스는 testing_ls에 넣어두었다.

In [29]:
name_ls = []
for idx,val in enumerate(tuned_df['title_bundle']):
    name_ls.append(val[1:testing_ls[idx]])
name_ls = [','.join(i) for i in name_ls]
# 리스트를 문자열 형식으로 바꿔준다.

In [30]:
tuned_df['author'] = name_ls

### 이번에는 저널명이다 testing_ls에서 1을 더해준다.

In [31]:
journal_ls = []
for idx,val in enumerate(tuned_df['title_bundle']):
    journal_ls.append(val[testing_ls[idx]+1])

In [32]:
tuned_df['journal'] = journal_ls

#### 나머지 4개의 요소를 보게 되면, 33(1)는 무엇인지 모르겠고 나머지는 각각 페이지 수와, 논문 발행 년도 , 논문의 분야 카테고리 , 마지막으로 피인용횟수가 되겠다. 각각 분배해주자.

In [33]:
tuned_df['title_bundle'].values[100]

['해외전환사채 전환권가치의 정보효과',
 '정무권 ',
 ' 한국증권학회 ',
 ' Asia-Pacific Journal of Financial Studies ',
 ' 33(1) ',
 ' pp.33~68 ',
 ' 2004.03 ',
 ' 경영학',
 '피인용횟수 : 3']

### 뭔지 모르겠지만 넣어준다.

In [34]:
dont_know_ls = []
for idx,val in enumerate(tuned_df['title_bundle']):
    dont_know_ls.append(val[-5])

tuned_df['dont_know'] = dont_know_ls

In [35]:
dont_know_ls[100]

' 33(1) '

In [36]:
[i for i in dont_know_ls if not re.findall(' [0-9]{2}\(.*\)',i)]

[]

### 페이지 수

In [37]:
page_ls = []
for idx,val in enumerate(tuned_df['title_bundle']):
    page_ls.append(val[-4])

tuned_df['page'] = page_ls

In [38]:
[i for i in page_ls if not re.findall(' pp.',i)]

[]

### 발행 시점

In [39]:
year_ls = []
for idx,val in enumerate(tuned_df['title_bundle']):
    year_ls.append(val[-3])

tuned_df['year'] = year_ls

In [40]:
[i for i in year_ls if not re.findall(' 20',i)]

[]

### 논문 분야

In [41]:
field_ls = []
for idx,val in enumerate(tuned_df['title_bundle']):
    field_ls.append(val[-2])

tuned_df['field'] = field_ls

In [42]:
[i for i in field_ls if not re.findall('경영학',i)]

[]

### 피인용 횟수

In [43]:
cited_ls = []
for idx,val in enumerate(tuned_df['title_bundle']):
    cited_ls.append(val[-1])

tuned_df['cited'] = cited_ls

In [44]:
[i for i in cited_ls if not re.findall('피인용횟수',i)]

[]

# 이렇게 해서 기존 new_df 라는 데이터 프레임에서 infor 컬럼의 튜닝을 끝냈다.

In [45]:
final_tuned_df = tuned_df.iloc[:,1:]
final_tuned_df.tail()

,paper_name,institution,author,journal,dont_know,page,year,field,cited
37242,KOSPI200 옵션전략지수의 성과,한국파생상품학회,"권순신 , 강병진 , 정재만",선물연구,26(2),pp.183~216,2018.05,경영학,피인용횟수 : 0
37243,KRX금시장과 국제금융시장의 상호연관성에 관한 연구 : CME선물시장을 중심으로,한국파생상품학회,"염명훈 , 김지훈",선물연구,26(3),pp.345~369,2018.08,경영학,피인용횟수 : 0
37244,Analytical Approximations of American Call Opt...,한국파생상품학회,배광일,선물연구,26(3),pp.283~310,2018.08,경영학,피인용횟수 : 0
37245,현금흐름 위험 기반 KOSPI 수익률 횡단면 연구,한국파생상품학회,"문성제 , 송준혁",선물연구,26(3),pp.311~343,2018.08,경영학,피인용횟수 : 0
37246,모호성 회피성향이 예방적 노력에 미치는 영향,한국파생상품학회,홍지민,선물연구,26(3),pp.371~389,2018.08,경영학,피인용횟수 : 0


In [46]:
final_tuned_df.to_csv('tuned_dataframe_infor.csv',index=False)

# 이번엔 refer 컬럼에 대한 튜닝을 해보자.

우선 / 로 split 이 가능해 보인다.

#### 참고문헌이 없어서 '0' 으로 등록되지 않은 리스트 중 첫 번째가   "숫자. [학술지 형태] - "  가 아닌 경우를 찾아본다.

In [47]:
new_refer_ls = []

for idx in range(len(new_df['refer'].values)):
    new_refer_ls.append(new_df['refer'].values[idx].split(' / '))
len(new_refer_ls) == len(new_df)

True

In [48]:
set([len(i) for idx,i in enumerate(new_refer_ls) if i[0] != '0'])

#3종류의 리스트가 있다.

{3, 4, 5}

In [49]:
for i in new_refer_ls : 
    if i[0] != '0' : 
        if not re.findall('[0-9]{1,}. \[.+\] -',i[0]) : 
            print(i)
            
# 없다. 참고문헌이 있는 리스트의 첫 번째 엘리먼트는 동일한 형태를 띄고 있음을 알 수 있다.

#### 참고문헌이 없어서 '0' 으로 등록되지 않은 리스트 중 두 번째가 년도의 형태가 아닌 경우를 찾아본다.

In [50]:
for i in new_refer_ls : 
    if i[0] != '0' : 
        if not re.findall('[0-9]{4}',i[1]) : 
            print(i)
            print(len(i))
            
# 안타깝게도 존재한다. 이런 경우 일단 귀속시킨 후에 교수님께 피드백을 요청한다. 7개만 나와서 그나마 다행인듯..ㅠㅠ

['4. [학술지(정기간행물)] -', '4', 'Shareholder Preferences and Dividend Policy (45) : 993 ~ 1018,']
3
['8. [학술지(정기간행물)] -', '4', 'Two Agency Cost Explanations of Dividends (74) : 650 ~ 659,']
3
['33. [학술지(정기간행물)] - Kyle, Albert S', '53', 'Continuous auctions and insider trading', 'Econometrica 53 : 1315 ~ 1335,']
4
['27. [보고서] - Huang, J', '203', 'How much of the corporate-treasury yield spread is due to credit risk : 1 ~ 57,']
3
['6. [학술지(정기간행물)] - Bliss, R. R.', '200', 'Testing the Stability of Implied Probability Density Functions', 'Journal of Banking and Finance 26 : 381 ~ 422,']
4
['16. [학술지(정기간행물)] - Figlewski, S.', '179', 'Subjective information and market efficiency in a betting market', 'Journal of Political Economy 87 : 75 ~ 89,']
4
['11. [학술지(정기간행물)] - Ait-Sahalia, Y.', '200', 'How Often to Sample a Continuous-Time Process in the Presence of Market Microstructure Noise', 'Review of Financial Studies 18 : 351 ~ 416,']
4


### 3개짜리의 구조
- 1. 논문의 성향
- 1.5. 논문의 저자(있는 경우도 있고 없는 경우도 있음)
- 2. 논문의 년도
- 3. 논문의 이름

In [51]:
len([i for i in new_refer_ls if len(i) == 3])

8071

### 즉, 3개짜리에는 발행기관명과 학술지명이 있는 경우, 없는 경우가 불규칙하게 분포되어 있다. 이 또한 교수님께 피드백을 받아야 하는 부분이다.

In [52]:
[i for i in new_refer_ls if len(i) == 3][0]

['1. [학술지(정기간행물)] -',
 '2001',
 '은행중심 자금조달 시스템의 문제점: 은행의 유동성위기가 기업가치에 미치는 영향 14 (1) : 161 ~ 188,']

In [53]:
print(len([i for i in new_refer_ls if len(i) == 3]))
[i[-1] for i in new_refer_ls if len(i) == 3][0]
# 즉, 3개짜리에는 발행기관명과 학술지명이 있는 경우, 없는 경우가 불규칙하게 분포되어 있다. 이 또한 교수님께 피드백을 받아야 하는 부분이다.

8071


'은행중심 자금조달 시스템의 문제점: 은행의 유동성위기가 기업가치에 미치는 영향 14 (1) : 161 ~ 188,'

In [54]:
len([j for j in [i[-1] for i in new_refer_ls if len(i) == 3] if re.findall('Journal',j)])
[j for j in [i[-1] for i in new_refer_ls if len(i) == 3] if re.findall('Journal',j)][:5]

['The Benefits of Lending Relationships Evidence from Small Business Data Journal of Finance 49 : 3 ~ 37,',
 'A Stylized Model of Customer Relationships Journal of Finance 45 : 1069 ~ 1087,',
 'The Value of Bank Durability Borrowers as Bank Stakeholders Journal of Finance 48 : 247 ~ 266,',
 'Valuation of primary issue convertible bonds Journal of Financial Research 9 : 251 ~ 259,',
 'The case of event studies Journal of Financial Economics 8 : 1 ~ 31,']

### 우선 이름이 있는 것을 뽑아주기 위해서 첫 번째 리스트를 한 번 더 세분화시켜준다.

In [55]:
tuned_new_refer = []
for idx in range(len(new_refer_ls)):
    tuned_new_refer.append(new_refer_ls[idx][0].split(" -") + new_refer_ls[idx][1:])
len(tuned_new_refer) == len(new_refer_ls)

True

In [56]:
merge = [(1, 3)]

for ls in tuned_new_refer : 
    if ls[0] != '0': 
        if not re.findall('[0-9]{1,4}',ls[2]) : 
            for t in merge[::-1]:
                merged = ''.join(ls[t[0]:t[1]])  # merging values within a range
                ls[t[0]:t[1]] = [merged]

In [57]:
for ls in tuned_new_refer : 
    if ls[0] != '0': 
        if not re.findall('[0-9]{1,4}',ls[2]) : 
            print(ls)

In [58]:
set([len(i) for idx,i in enumerate(tuned_new_refer) if i[0] != '0'])

#3종류의 리스트가 있다.

{4, 5, 6}

## 길이가 4개짜리인 데이터 프레임만들기

In [59]:
[i for i in tuned_new_refer if len(i) == 4][0]

['1. [학술지(정기간행물)]',
 '',
 '2001',
 '은행중심 자금조달 시스템의 문제점: 은행의 유동성위기가 기업가치에 미치는 영향 14 (1) : 161 ~ 188,']

## Characteristic of paper

In [60]:
char_paper_ls = [re.findall('\[.+\]',i[0])[0] for i in [i for i in tuned_new_refer if len(i) == 4]]

## author of paper

In [61]:
name_ls = [i[1] for i in tuned_new_refer if len(i) == 4]

## year of paper published

In [62]:
year_ls = [i[2] for i in tuned_new_refer if len(i) == 4]

## name of paper

In [63]:
paper_ls = [i[3] for i in tuned_new_refer if len(i) == 4]

In [64]:
len(char_paper_ls) == len(name_ls) == len(year_ls) == len(paper_ls)

True

In [65]:
len_4_df = pd.DataFrame()
len_4_df['char'] = char_paper_ls
len_4_df['name'] = name_ls
len_4_df['year'] = year_ls
len_4_df['paper'] = paper_ls

len_4_df.tail()

,char,name,year,paper
8066,[보고서],"Fink, J.",2006,"Firm age and fluctuations in idiosyncratic risk,"
8067,[보고서],"Liu, B.",2012,Idiosyncratic volatility on Australian stock r...
8068,[보고서],"Chen, R. R.",2005,"Dynamic Interactions between Interest rate, Cr..."
8069,[보고서],"Kim, J.",2017,A Theory on the Expected Returns of Defautable...
8070,[보고서],"Tang, D. Y.",2007,"Liquidity and Credit Default Swap Spreads,"


---------------

# 4개짜리의 구조
- 마지막 엘리먼트는 저널명이 되겠다.

In [66]:
[i for i in new_refer_ls if len(i) == 4][800]

['9. [학술지(정기간행물)] - Kreps, D',
 '1982',
 'Reputation and Imperfect Information',
 'Journal of Economic Theory 27 : 253 ~ 279,']

In [67]:
list(set([paper[-1] for paper in [i for i in new_refer_ls if len(i) == 4]]))[-5:]

['Journal of Finance 63 : 2345 ~ 2384,',
 '한국경제연구 28 (4) : 39 ~ 90,',
 '보험학회지 76 (76) : 32 ~ 60,',
 'Review of Financial Studies 20 (5) : 1429 ~ 1460,',
 'Journal of Financial Research 5 : 95 ~ 104,']

In [68]:
print(len([paper[-1] for paper in [i for i in new_refer_ls if len(i) == 4]]))

28840


- 아래는 길이가 4개인 리스트에서 저널명을 나타내는 마지막 리스트를 빼온 것이다.
- 이 리스트를 통해서 길이가 3개짜리인 리스트의 마지막에서 저널명이 포함된 것을 빼와본다.
- 우선 일반화를 위해서 숫자를 다 빼고 유니크한 값으로 변환시켜준다.

In [69]:
[paper[-1] for paper in [i for i in new_refer_ls if len(i) == 4]][-5:]

['Journal of Finance 58 (3) : 1113 ~ 1137,',
 'Journal of Political Economy 111 (6) : 1183 ~ 1220,',
 'Journal of Finance 52 (1) : 35 ~ 55,',
 'Journal of Financial and Quantitative Analysis 34 (1) : 89 ~ 114,',
 'Journal of Business 75 (4) : 583 ~ 608']

In [70]:
len([paper[-1] for paper in [i for i in new_refer_ls if len(i) == 4]])

28840

좀 걸린다.

# 4개짜리 데이터 프레임으로 만들기

In [71]:
[i for i in tuned_new_refer if len(i) == 5][0]

['4. [학술지(정기간행물)]',
 ' 정완호',
 '2003',
 '기업 신용위험의 측정- 채권수익률을 이용하여 -',
 'Asia-Pacific Journal of Financial Studies 32 (1) : 135 ~ 172,']

## Characteristic of paper

In [72]:
char_paper_ls = [re.findall('\[.+\]',i[0])[0] for i in [i for i in tuned_new_refer if len(i) == 5]]

## author of paper

In [73]:
name_ls = [i[1] for i in tuned_new_refer if len(i) == 5]

## year of paper published

In [74]:
year_ls = [i[2] for i in tuned_new_refer if len(i) == 5]

## name of paper

In [75]:
paper_ls = [i[3] for i in tuned_new_refer if len(i) == 5]

## journal name which paper published

In [76]:
journal_ls = [i[-1] for i in tuned_new_refer if len(i) == 5]

In [77]:
len(char_paper_ls) == len(name_ls) == len(year_ls) == len(paper_ls) == len(journal_ls)

True

In [78]:
len_5_df = pd.DataFrame()
len_5_df['char'] = char_paper_ls
len_5_df['name'] = name_ls
len_5_df['year'] = year_ls
len_5_df['paper'] = paper_ls
len_5_df['journal'] = journal_ls

len_5_df.tail()

,char,name,year,paper,journal
28835,[학술지(정기간행물)],"Ofek, E.",2003,Dotcom Mania : The Rise and Fall of Internet S...,"Journal of Finance 58 (3) : 1113 ~ 1137,"
28836,[학술지(정기간행물)],"Scheinkman, J. A.",2003,Overconfidence and Speculative Bubbles,Journal of Political Economy 111 (6) : 1183 ~ ...
28837,[학술지(정기간행물)],"Shleifer, A.",1997,The Limits of Arbitrage,"Journal of Finance 52 (1) : 35 ~ 55,"
28838,[학술지(정기간행물)],"Tkac, P. A.",1999,A Trading Volume Benchmark : Theory and Evidence,Journal of Financial and Quantitative Analysis...
28839,[학술지(정기간행물)],"Wurgler, J.",2002,Does Arbitrage Flatten Demand Curves for Stocks?,Journal of Business 75 (4) : 583 ~ 608


-------------------------------------

# 5개짜리의 구조

In [79]:
len([i for i in new_refer_ls if len(i) == 5])

94

- 우선 첫 번째 리스트에서 - 이후에 성명이 나와있는 경우가 있다.
- 마지막과 그 전 리스트 엘리먼트는 해당 저자의 소속과 학위가 나와있다.

# 학위

In [80]:
set([paper[-2] for paper in [i for i in new_refer_ls if len(i) == 5]])

{'Ph.D', '博士', '박사', '석사'}

# 소속

In [81]:
list(set([paper[-1] for paper in [i for i in new_refer_ls if len(i) == 5]]))[-5:]

['UZH and ETHZ,',
 '전남대학교 대학원,',
 '계명대학교 대학원,',
 'Seoul National University,',
 '명지대대학원,']

# 길이가 6개인 리스트를 나눠서 데이터 프레임으로 넣기

In [82]:
[i for i in tuned_new_refer if len(i) == 6][0]

['22. [학위논문]',
 ' Low, A',
 '1992',
 'Essays on asymmetric information in international finance',
 '박사',
 'University of California,']

## Characteristic of paper

In [83]:
char_paper_ls = [re.findall('\[.+\]',i[0])[0] for i in [i for i in tuned_new_refer if len(i) == 6]]

## Author of paper

In [84]:
name_ls = []

for k in range(len([i for i in tuned_new_refer if len(i) == 6])):
    name_ls.append([i for i in tuned_new_refer if len(i) == 6][k]\
[1:[idx for idx,val in enumerate([i for i in tuned_new_refer if len(i) == 6][k]) if re.findall("[0-9]{4}",val)][0]][0])

## Year of paper published

In [85]:
year_ls = []

for k in range(len([i for i in tuned_new_refer if len(i) == 6])):
    year_ls.append([i for i in tuned_new_refer if len(i) == 6][k]\
[[idx for idx,val in enumerate([i for i in tuned_new_refer if len(i) == 6][k]) if re.findall("[0-9]{4}",val)][0]])

## Name of paper

In [86]:
paper_ls = []

for k in range(len([i for i in tuned_new_refer if len(i) == 6])):
    paper_ls.append([i for i in tuned_new_refer if len(i) == 6][k]\
[[idx+1 for idx,val in enumerate([i for i in tuned_new_refer if len(i) == 6][k]) if re.findall("[0-9]{4}",val)][0]])

## Details about Author (degree , school)

In [87]:
degree_ls = [i[-2] for i in tuned_new_refer if len(i) == 6]
school_ls = [i[-1] for i in tuned_new_refer if len(i) == 6]

In [88]:
len(char_paper_ls) == len(name_ls) == len(year_ls) == len(paper_ls) == len(degree_ls) == len(school_ls)

True

In [89]:
len_6_df = pd.DataFrame()
len_6_df['char'] = char_paper_ls
len_6_df['name'] = name_ls
len_6_df['year'] = year_ls
len_6_df['paper'] = paper_ls
len_6_df['degree'] = degree_ls
len_6_df['school'] = school_ls

len_6_df.tail()

,char,name,year,paper,degree,school
89,[학위논문],"Wei, X.",2009,The price effect and the issuance clustering a...,석사,"National University of Singapore,"
90,[학위논문],박성환,2011,지방채발행의 추이분석과 영향구조에 관한 연구,박사,"영남대학교 대학원,"
91,[학위논문],김동현,2008,국내 민간의료보험 시장의 역선택에 관한 실증연구,석사,"경희대학교 대학원,"
92,[학위논문],이수진,2012,민간의료보험이 의료수요에 미치는 영향 : 도덕적 해이와 역선택 효과의 실증적 구분,석사,"연세대학교 대학원,"
93,[학위논문],한영하,2014,한국주택금융공사 보금자리론 조기상환율 추정 및 가치산정 연구,박사,"연세대학교 대학원,"


# concatenate the 3 dataframe

In [90]:
final_df = pd.concat([len_4_df,len_5_df],axis=0)
final_df = pd.concat([final_df,len_6_df],axis=0)
final_df.reset_index(drop=True,inplace=True)
final_df.tail()

,char,degree,journal,name,paper,school,year
37000,[학위논문],석사,NaN,"Wei, X.",The price effect and the issuance clustering a...,"National University of Singapore,",2009
37001,[학위논문],박사,NaN,박성환,지방채발행의 추이분석과 영향구조에 관한 연구,"영남대학교 대학원,",2011
37002,[학위논문],석사,NaN,김동현,국내 민간의료보험 시장의 역선택에 관한 실증연구,"경희대학교 대학원,",2008
37003,[학위논문],석사,NaN,이수진,민간의료보험이 의료수요에 미치는 영향 : 도덕적 해이와 역선택 효과의 실증적 구분,"연세대학교 대학원,",2012
37004,[학위논문],박사,NaN,한영하,한국주택금융공사 보금자리론 조기상환율 추정 및 가치산정 연구,"연세대학교 대학원,",2014


In [91]:
final_df.to_csv('tuned_dataframe_refer.csv',index=False)

# concatenate tuned_infor and tuned_refer dataframe

In [92]:
infor_csv = pd.read_csv("tuned_dataframe_infor.csv")
refer_csv = pd.read_csv("tuned_dataframe_refer.csv")

df = pd.concat([infor_csv,refer_csv],axis=1)

In [93]:
df.columns = ['paper_name_infor', 'institution_infor', 'author_infor', 'journal_infor', 'dont_know_infor', 'page_infor',
       'year_infor', 'field_infor', 'cited_infor', 'char_refer', 'degree_refer', 'journal_refer', 'name_refer', 'paper_refer',
       'school_refer', 'year_refer']

In [94]:
df.to_csv('tuned_dataframe.csv',index=False)

In [95]:
df.tail()

,paper_name_infor,institution_infor,author_infor,journal_infor,dont_know_infor,page_infor,year_infor,field_infor,cited_infor,char_refer,degree_refer,journal_refer,name_refer,paper_refer,school_refer,year_refer
37242,KOSPI200 옵션전략지수의 성과,한국파생상품학회,"권순신 , 강병진 , 정재만",선물연구,26(2),pp.183~216,2018.05,경영학,피인용횟수 : 0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37243,KRX금시장과 국제금융시장의 상호연관성에 관한 연구 : CME선물시장을 중심으로,한국파생상품학회,"염명훈 , 김지훈",선물연구,26(3),pp.345~369,2018.08,경영학,피인용횟수 : 0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37244,Analytical Approximations of American Call Opt...,한국파생상품학회,배광일,선물연구,26(3),pp.283~310,2018.08,경영학,피인용횟수 : 0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37245,현금흐름 위험 기반 KOSPI 수익률 횡단면 연구,한국파생상품학회,"문성제 , 송준혁",선물연구,26(3),pp.311~343,2018.08,경영학,피인용횟수 : 0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37246,모호성 회피성향이 예방적 노력에 미치는 영향,한국파생상품학회,홍지민,선물연구,26(3),pp.371~389,2018.08,경영학,피인용횟수 : 0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 정리
- 4개짜리 리스트 (8071개)
    - 논문의 특성
    - 이름(공백이라고 해도 무방)
    - 년도
    - 논문 이름(저널이름이 함께 포함되어 있는 경우가 있음, Journal만 있는 경우가 900개 넘는다.)
- 5개짜리 리스트 (28840개)
    - 논문의 특성
    - 이름
    - 년도
    - 논문의 이름
    - 저널의 이름
- 6개짜리 리스트 (94개)
    - 논문의 특성
    - 이름
    - 년도
    - 논문의 이름
    - 저자의 학위
    - 저자의 소속

In [96]:
(len([i for i in tuned_new_refer if len(i) == 4]),
len([i for i in tuned_new_refer if len(i) == 5]),
len([i for i in tuned_new_refer if len(i) == 6]))

(8071, 28840, 94)

In [97]:
[i for i in tuned_new_refer if len(i) == 4][0]

['1. [학술지(정기간행물)]',
 '',
 '2001',
 '은행중심 자금조달 시스템의 문제점: 은행의 유동성위기가 기업가치에 미치는 영향 14 (1) : 161 ~ 188,']

In [98]:
[i for i in tuned_new_refer if len(i) == 5][0]

['4. [학술지(정기간행물)]',
 ' 정완호',
 '2003',
 '기업 신용위험의 측정- 채권수익률을 이용하여 -',
 'Asia-Pacific Journal of Financial Studies 32 (1) : 135 ~ 172,']

In [99]:
[i for i in tuned_new_refer if len(i) == 6][0]

['22. [학위논문]',
 ' Low, A',
 '1992',
 'Essays on asymmetric information in international finance',
 '박사',
 'University of California,']

# ! 34,50,52 부분을 교수님께 여쭈어보아야 한다.

## 추가적인 튜닝 (아직 적용하지는 않았음)

In [ ]:
extract_journal_ls = []
for idx in range(28840):
    extract_journal_ls.append(\
','.join(re.findall('\D',[paper[-1] for paper in [i for i in new_refer_ls if len(i) == 4]][idx]))\
.replace(',','').split(' : ')[0])
                              
len(extract_journal_ls) == len([paper[-1] for paper in [i for i in new_refer_ls if len(i) == 4]])

In [ ]:
extract_journal_ls = [i.replace('(','') for i in extract_journal_ls]
extract_journal_ls = [i.replace(')','') for i in extract_journal_ls]
print(len(extract_journal_ls))
extract_journal_ls = list(set(extract_journal_ls))
print(len(extract_journal_ls))